In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("vectorbt_reports/new_expanding_master_backtest_stats.csv")
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna(subset=['Sharpe Ratio'])
df = df.drop(columns=['Start Value', 'End Value'])
#df = df[df["asset"]=="NVDA"]
#display(df[df["asset"]=="NVDA"])

In [4]:
returns = df['Total Return [%]'] / 100
cumulative_return = (1 + returns).prod() - 1
N = len(returns)  # number of periods
annualized_return = (1 + cumulative_return)**(12 / N) - 1
annualized_volatility = returns.std() * np.sqrt(12)
sharpe_annual = annualized_return / annualized_volatility
print(sharpe_annual)

-0.9182679985993166


In [5]:
def compute_annualized_sharpe(df):
    results = []

    for asset, group in df.groupby("asset"):
        # Convert returns from % to decimals
        returns = group['Total Return [%]'] / 100
        benchmark_returns = group['Benchmark Return [%]'] / 100
        N = len(returns)

        if N < 2:
            continue  # Skip assets with insufficient data

        # Strategy Sharpe
        cumulative_return = (1 + returns).prod() - 1
        annualized_return = (1 + cumulative_return)**(12 / N) - 1
        annualized_volatility = returns.std() * np.sqrt(12)
        sharpe_strategy = annualized_return / annualized_volatility if annualized_volatility > 0 else np.nan

        # Benchmark Sharpe
        cumulative_bench = (1 + benchmark_returns).prod() - 1
        annualized_bench_return = (1 + cumulative_bench)**(12 / N) - 1
        annualized_bench_vol = benchmark_returns.std() * np.sqrt(12)
        sharpe_benchmark = annualized_bench_return / annualized_bench_vol if annualized_bench_vol > 0 else np.nan

        results.append({
            'asset': asset,
            'strategy_sharpe': sharpe_strategy,
            'benchmark_sharpe': sharpe_benchmark
        })

    return pd.DataFrame(results)

In [6]:
df = pd.read_csv("vectorbt_reports/new_expanding_master_backtest_stats.csv")
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna(subset=['Sharpe Ratio'])
df = df.drop(columns=['Start Value', 'End Value'])
sharpe_df = compute_annualized_sharpe(df)
display(sharpe_df.sort_values(by='strategy_sharpe', ascending=False))

,asset,strategy_sharpe,benchmark_sharpe
3,BTC=F,0.346582,0.571205
9,GC=F,0.017298,2.864663
0,AAPL,-0.348613,0.652176
13,NVDA,-0.449081,-0.209887
8,ES=F,-0.739455,0.789796
12,MSFT,-0.877117,-0.405248
14,TSLA,-0.913867,0.965867
1,AMZN,-1.098914,0.382994
2,BTC-USD,-1.246533,0.647465
5,CL=F,-1.574279,-0.187782


In [7]:
for asset_name in df["asset"].unique():
    asset_df = df[df["asset"] == asset_name]
    print(f"Asset: {asset_name}")
    display(asset_df)

Asset: AAPL


,Start,End,Period,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],Max Drawdown Duration,Total Trades,...,Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,asset
0,2024-06-03,2024-07-01,20 days,0.918178,11.709530,100.0,30.193849,3.419461,9 days,2,...,NaN,6 days,NaN,NaN,101.909621,0.958649,5.308478,1.231398,1.636180,AAPL
15,2024-07-01 00:00:00,2024-07-31 00:00:00,22 days 00:00:00,1.927529,2.459055,100.0,41.149418,5.078182,15 days 00:00:00,2,...,-2.727069,7 days 00:00:00,1 days 00:00:00,1.675230,96.376447,1.639116,7.338482,1.387341,2.315795,AAPL
30,2024-07-31 00:00:00,2024-08-30 00:00:00,23 days 00:00:00,5.795053,3.115993,100.0,9.990010,0.874132,2 days 00:00:00,1,...,NaN,NaN,NaN,NaN,NaN,7.455835,165.291295,3.753466,20.062507,AAPL
45,2024-08-30 00:00:00,2024-09-27 00:00:00,20 days 00:00:00,0.108406,-0.528387,100.0,29.407052,2.914887,12 days 00:00:00,2,...,-2.917802,NaN,1 days 00:00:00,0.000000,-291.488674,0.193160,0.685110,1.049328,0.319999,AAPL
60,2024-09-30 00:00:00,2024-10-29 00:00:00,22 days 00:00:00,-4.554933,0.287553,100.0,58.094472,4.459488,21 days 00:00:00,3,...,-1.536853,NaN,3 days 08:00:00,0.000000,-151.831106,-4.299635,-12.077278,0.405864,-4.699217,AAPL
75,2024-10-29 00:00:00,2024-11-27 00:00:00,22 days 00:00:00,-0.373799,0.539220,100.0,49.023439,3.304101,15 days 00:00:00,3,...,-1.618389,NaN,1 days 12:00:00,0.000000,-160.370249,-0.504156,-1.823247,0.906042,-0.681076,AAPL
90,2024-11-29 00:00:00,2024-12-27 00:00:00,20 days 00:00:00,-0.139150,7.693926,100.0,59.587302,2.337698,7 days 00:00:00,3,...,-0.847556,3 days 00:00:00,2 days 00:00:00,0.917623,-4.638344,-0.145637,-1.073382,0.968650,-0.181422,AAPL
105,2024-12-30 00:00:00,2025-01-27 00:00:00,18 days 00:00:00,-5.003613,-8.858048,100.0,47.648996,7.929631,8 days 00:00:00,3,...,-3.812185,NaN,1 days 00:00:00,0.000000,-373.675464,-3.560977,-8.157480,0.391501,-4.195704,AAPL
120,2025-01-27 00:00:00,2025-02-26 00:00:00,22 days 00:00:00,-3.182863,4.567998,100.0,29.091384,4.932140,19 days 00:00:00,2,...,-4.497721,NaN,4 days 00:00:00,0.000000,-449.322785,-2.404391,-8.420233,0.611939,-2.790128,AAPL
135,2025-02-26 00:00:00,2025-03-28 00:00:00,23 days 00:00:00,1.134690,-9.344320,100.0,9.990010,2.658035,2 days 00:00:00,1,...,NaN,NaN,NaN,NaN,NaN,1.167771,7.377184,1.282826,1.669691,AAPL


Asset: AMZN


,Start,End,Period,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],Max Drawdown Duration,Total Trades,...,Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,asset
1,2024-06-03 00:00:00,2024-07-01 00:00:00,20 days 00:00:00,0.713331,10.575306,100.0,50.393670,2.520152,10 days 00:00:00,3,...,-0.385755,6 days 00:00:00,2 days 00:00:00,3.087242,40.702241,0.750593,5.496120,1.161907,1.151314,AMZN
16,2024-07-01 00:00:00,2024-07-31 00:00:00,22 days 00:00:00,-5.756483,-5.182556,100.0,47.427293,9.604633,18 days 00:00:00,3,...,-4.043261,NaN,6 days 00:00:00,0.000000,-396.127471,-3.780515,-6.518251,0.524538,-4.597819,AMZN
31,2024-07-31 00:00:00,2024-08-30 00:00:00,23 days 00:00:00,4.753382,-4.535242,100.0,9.990010,5.169062,7 days 00:00:00,1,...,NaN,NaN,NaN,NaN,NaN,2.875966,21.078804,1.777856,6.324259,AMZN
46,2024-08-30 00:00:00,2024-09-27 00:00:00,20 days 00:00:00,0.677099,5.305323,100.0,58.726648,5.248156,12 days 00:00:00,3,...,-2.654767,12 days 00:00:00,1 days 00:00:00,1.129017,22.569978,0.602662,2.497235,1.101709,0.852706,AMZN
61,2024-09-30 00:00:00,2024-10-29 00:00:00,22 days 00:00:00,-3.688322,2.415070,100.0,38.979803,3.688322,18 days 00:00:00,2,...,-1.853300,NaN,5 days 12:00:00,0.000000,-184.416122,-3.225088,-12.578434,0.492113,-3.498396,AMZN
76,2024-10-29 00:00:00,2024-11-27 00:00:00,22 days 00:00:00,0.915973,7.813239,100.0,40.523720,6.612463,10 days 00:00:00,2,...,-1.218895,10 days 00:00:00,1 days 00:00:00,1.736326,45.798670,0.651238,2.469856,1.123391,0.978766,AMZN
91,2024-11-29 00:00:00,2024-12-27 00:00:00,20 days 00:00:00,-5.527110,7.629035,100.0,39.444621,6.759190,11 days 00:00:00,2,...,-2.766616,NaN,3 days 12:00:00,0.000000,-276.355494,-4.374749,-9.553097,0.295277,-4.836034,AMZN
106,2024-12-30 00:00:00,2025-01-27 00:00:00,18 days 00:00:00,4.113390,6.380477,100.0,29.764883,3.784324,8 days 00:00:00,2,...,-1.126857,NaN,2 days 00:00:00,0.000000,-112.573176,3.789893,33.417244,1.997563,6.670516,AMZN
121,2025-01-27 00:00:00,2025-02-26 00:00:00,22 days 00:00:00,-4.447142,-8.949958,100.0,48.814634,6.761454,15 days 00:00:00,3,...,-2.541856,NaN,2 days 00:00:00,0.000000,-252.161971,-3.366547,-7.836519,0.378436,-3.689622,AMZN
136,2025-02-26 00:00:00,2025-03-28 00:00:00,23 days 00:00:00,-5.466335,-10.090975,100.0,48.650874,6.314717,9 days 00:00:00,3,...,-2.013381,NaN,1 days 00:00:00,0.000000,-199.357281,-3.117353,-9.346474,0.507893,-3.805914,AMZN


Asset: BTC-USD


...

,Start,End,Period,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],Max Drawdown Duration,Total Trades,...,Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,asset
12,2024-06-03 00:00:00,2024-07-01 00:00:00,20 days 00:00:00,-1.009494,10.449319,100.0,70.013283,2.526008,9 days 00:00:00,4,...,-0.877388,5 days 00:00:00,2 days 00:00:00,0.482850,-30.350101,-1.965305,-6.691823,0.676347,-2.633020,MSFT
27,2024-07-01 00:00:00,2024-07-31 00:00:00,22 days 00:00:00,-2.455389,-8.403215,100.0,39.625580,4.619507,18 days 00:00:00,2,...,-1.234795,NaN,5 days 00:00:00,0.000000,-122.769443,-2.427887,-7.316314,0.633520,-2.957696,MSFT
42,2024-07-31 00:00:00,2024-08-30 00:00:00,23 days 00:00:00,3.484886,-0.289229,100.0,9.990010,3.342745,8 days 00:00:00,1,...,NaN,NaN,NaN,NaN,NaN,3.760844,21.605800,1.782770,5.806399,MSFT
57,2024-08-30 00:00:00,2024-09-27 00:00:00,20 days 00:00:00,2.319937,2.608231,100.0,39.823862,2.042018,6 days 00:00:00,2,...,-2.044060,9 days 00:00:00,1 days 00:00:00,2.136100,115.996838,2.496080,25.453217,1.536278,4.262500,MSFT
72,2024-09-30 00:00:00,2024-10-29 00:00:00,22 days 00:00:00,1.177241,0.383459,100.0,49.045811,2.428660,15 days 00:00:00,3,...,-2.431089,6 days 00:00:00,1 days 00:00:00,0.056051,-114.626534,1.307229,8.824119,1.254337,1.948279,MSFT
87,2024-10-29 00:00:00,2024-11-27 00:00:00,22 days 00:00:00,-0.114121,-2.074319,100.0,29.704002,2.979499,9 days 00:00:00,2,...,-1.431570,NaN,7 days 00:00:00,0.000000,-143.014032,-0.013479,-0.629844,0.997493,-0.018230,MSFT
102,2024-11-29 00:00:00,2024-12-27 00:00:00,20 days 00:00:00,1.284537,1.669581,100.0,20.128602,3.852345,7 days 00:00:00,1,...,NaN,12 days 00:00:00,NaN,NaN,128.453696,1.227794,6.808828,1.274364,1.509102,MSFT
117,2024-12-30 00:00:00,2025-01-27 00:00:00,18 days 00:00:00,0.227885,2.290331,100.0,39.727238,2.817164,10 days 00:00:00,2,...,-1.479544,7 days 00:00:00,1 days 00:00:00,1.154178,11.394248,0.305983,1.676838,1.071664,0.583565,MSFT
132,2025-01-27 00:00:00,2025-02-26 00:00:00,22 days 00:00:00,-8.936665,-8.015001,100.0,46.743325,8.936665,21 days 00:00:00,3,...,-4.485421,NaN,3 days 00:00:00,0.000000,-442.280089,-5.508085,-8.822318,0.147725,-5.505894,MSFT
147,2025-02-26 00:00:00,2025-03-28 00:00:00,23 days 00:00:00,-5.917820,-5.236040,100.0,48.850097,5.917820,18 days 00:00:00,3,...,-1.808237,NaN,2 days 00:00:00,0.000000,-179.060210,-5.994667,-10.479936,0.279162,-6.181132,MSFT


Asset: NVDA


,Start,End,Period,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],Max Drawdown Duration,Total Trades,...,Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,asset
13,2024-06-03 00:00:00,2024-07-01 00:00:00,20 days 00:00:00,10.182755,8.086959,100.0,42.157886,3.636807,8 days 00:00:00,2,...,NaN,5 days 00:00:00,NaN,NaN,509.137729,5.133738,133.884726,2.736209,11.044452,NVDA
28,2024-07-01 00:00:00,2024-07-31 00:00:00,22 days 00:00:00,-8.296992,-5.856803,100.0,28.348970,12.716440,15 days 00:00:00,2,...,-8.213495,NaN,9 days 00:00:00,0.000000,-820.528942,-3.258091,-5.995118,0.510767,-3.642725,NVDA
43,2024-07-31 00:00:00,2024-08-30 00:00:00,23 days 00:00:00,-0.149649,2.008209,100.0,57.164681,9.636611,9 days 00:00:00,3,...,-3.638386,13 days 00:00:00,1 days 00:00:00,0.979334,-4.988297,0.252746,-0.243719,1.040240,0.358214,NVDA
58,2024-08-30 00:00:00,2024-09-27 00:00:00,20 days 00:00:00,12.191235,1.700594,100.0,9.990010,4.843039,7 days 00:00:00,1,...,NaN,NaN,NaN,NaN,NaN,4.699536,147.864435,2.384347,13.088979,NVDA
73,2024-09-30 00:00:00,2024-10-29 00:00:00,22 days 00:00:00,-0.373924,16.312580,100.0,59.751701,7.715789,11 days 00:00:00,3,...,-3.468675,8 days 00:00:00,1 days 12:00:00,0.946708,-12.464139,0.008847,-0.781016,1.001637,0.011881,NVDA
88,2024-10-29 00:00:00,2024-11-27 00:00:00,22 days 00:00:00,-7.677820,-4.184073,100.0,58.082555,8.192913,14 days 00:00:00,3,...,-2.626611,NaN,3 days 16:00:00,0.000000,-255.927327,-5.085862,-8.962592,0.400461,-5.782243,NVDA
103,2024-11-29 00:00:00,2024-12-27 00:00:00,20 days 00:00:00,-6.536682,-0.896930,100.0,55.785659,10.765548,17 days 00:00:00,3,...,-5.479869,5 days 00:00:00,3 days 00:00:00,0.387740,-217.889391,-3.785487,-6.583901,0.524355,-5.073158,NVDA
118,2024-12-30 00:00:00,2025-01-27 00:00:00,18 days 00:00:00,-3.260893,-13.870104,100.0,38.392757,6.497834,14 days 00:00:00,2,...,-6.410737,4 days 00:00:00,1 days 00:00:00,0.490830,-163.044636,-1.487114,-7.532441,0.698572,-1.863993,NVDA
133,2025-01-27 00:00:00,2025-02-26 00:00:00,22 days 00:00:00,1.874243,10.859653,100.0,29.383203,9.621014,5 days 00:00:00,2,...,-3.037167,NaN,1 days 00:00:00,0.000000,-303.413256,0.956559,3.750169,1.154811,1.422633,NVDA
148,2025-02-26 00:00:00,2025-03-28 00:00:00,23 days 00:00:00,-5.813554,-16.461000,100.0,39.324682,10.314454,10 days 00:00:00,2,...,-2.922242,NaN,5 days 00:00:00,0.000000,-290.677701,-2.255046,-5.947464,0.626857,-2.995122,NVDA


Asset: TSLA


,Start,End,Period,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],Max Drawdown Duration,Total Trades,...,Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,asset
14,2024-06-03 00:00:00,2024-07-01 00:00:00,20 days 00:00:00,0.488885,19.042492,100.0,48.286895,5.970639,11 days 00:00:00,3,...,-2.985166,NaN,1 days 12:00:00,0.000000,-293.830472,0.432388,1.559048,1.119098,0.898631,TSLA
29,2024-07-01 00:00:00,2024-07-31 00:00:00,22 days 00:00:00,-6.410628,10.583249,100.0,67.422393,15.890714,15 days 00:00:00,4,...,-2.149310,NaN,2 days 16:00:00,0.000000,-210.164279,-1.965242,-4.196562,0.680048,-2.343665,TSLA
44,2024-07-31 00:00:00,2024-08-30 00:00:00,23 days 00:00:00,-4.018732,-7.739047,100.0,28.641876,9.635062,14 days 00:00:00,2,...,-6.747506,NaN,1 days 00:00:00,0.000000,-674.076483,-0.891396,-4.965577,0.863628,-1.235985,TSLA
59,2024-08-30 00:00:00,2024-09-27 00:00:00,20 days 00:00:00,0.288364,21.647747,100.0,88.303082,8.628858,16 days 00:00:00,5,...,-2.994907,5 days 12:00:00,1 days 12:00:00,1.065788,9.716319,0.365256,0.625298,1.070471,0.527099,TSLA
74,2024-09-30 00:00:00,2024-10-29 00:00:00,22 days 00:00:00,5.712078,-0.806488,100.0,37.955441,16.014738,14 days 00:00:00,2,...,-13.094961,6 days 00:00:00,5 days 00:00:00,1.436641,285.603920,1.350615,9.449598,1.447233,3.356529,TSLA
89,2024-10-29 00:00:00,2024-11-27 00:00:00,22 days 00:00:00,-6.622523,28.271435,100.0,58.158280,14.215120,12 days 00:00:00,3,...,-3.740472,2 days 00:00:00,2 days 12:00:00,0.100993,-220.750766,-1.805467,-4.777722,0.660514,-2.564387,TSLA
104,2024-11-29 00:00:00,2024-12-27 00:00:00,20 days 00:00:00,-5.133550,25.060841,100.0,19.486151,8.371215,7 days 00:00:00,1,...,-5.138684,NaN,2 days 00:00:00,0.000000,-513.355046,-2.220836,-7.379893,0.429340,-2.466776,TSLA
119,2024-12-30 00:00:00,2025-01-27 00:00:00,18 days 00:00:00,-13.951135,-4.853743,100.0,74.594251,13.865087,17 days 00:00:00,4,...,-3.682883,NaN,2 days 12:00:00,0.000000,-348.778386,-9.297537,-6.869708,0.179151,-9.315556,TSLA
134,2025-01-27 00:00:00,2025-02-26 00:00:00,22 days 00:00:00,-10.362668,-26.778297,100.0,37.890732,10.362668,18 days 00:00:00,2,...,-5.328322,NaN,3 days 00:00:00,0.000000,-518.133415,-5.880187,-8.078662,0.146101,-5.914995,TSLA
149,2025-02-26 00:00:00,2025-03-28 00:00:00,23 days 00:00:00,16.734181,-9.370702,100.0,40.824187,4.978617,4 days 00:00:00,2,...,-4.258979,4 days 00:00:00,3 days 00:00:00,4.933082,836.709064,4.375748,213.956756,3.052660,11.524186,TSLA


In [4]:
df = pd.read_csv("vectorbt_reports/fixed_master_backtest_stats_all_assets.csv")
display(df)


,Start,End,Period,Start Value,End Value,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],...,Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,asset
0,2024-04-01,2025-04-01,252 days,10000.0,10548.556943,5.485569,31.265073,100.0,368.458634,10.650710,...,-2.491261,16 days 09:00:00,1 days 21:20:00,1.210514,29.938652,0.479648,0.755079,1.090375,0.627710,AAPL
1,2024-04-01 00:00:00,2025-04-01 00:00:00,252 days 00:00:00,10000.0,11120.363393,11.203634,6.188869,100.0,343.169904,14.061495,...,-3.021446,15 days 20:00:00,2 days 14:24:00,1.351225,70.022712,0.713708,1.182453,1.155763,1.005632,AMZN
2,2024-04-01 00:00:00,2025-04-01 00:00:00,366 days 00:00:00,10000.0,7560.433232,-24.395668,22.190167,100.0,580.908626,27.273431,...,-2.389496,7 days 00:00:00,2 days 19:23:04.615384615,0.571324,-76.236462,-0.876407,-0.892366,0.805031,-1.232373,BTC-USD
3,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,9033.107670,-9.668923,21.270247,100.0,444.570150,22.759254,...,-2.768439,10 days 00:00:00,2 days 09:52:56.470588235,0.779633,-40.287180,-0.212025,-0.599542,0.959792,-0.314476,BTC=F
4,2024-04-01 00:00:00,2025-03-31 00:00:00,260 days 00:00:00,10000.0,9526.674376,-4.733256,2.404336,100.0,424.169364,8.633697,...,-0.728519,13 days 04:00:00,3 days 16:00:00,0.595916,-20.838842,-0.971693,-0.762205,0.835588,-1.367147,CHFUSD=X
5,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,8856.059554,-11.439404,-14.944454,100.0,313.604272,21.638989,...,-3.194777,10 days 10:17:08.571428571,6 days 12:00:00,0.495337,-88.302062,-0.492570,-0.742933,0.919112,-0.666829,CL=F
6,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,10249.784241,2.497842,-0.723666,100.0,276.865293,2.824967,...,-0.541636,20 days 14:24:00,2 days 00:00:00,1.574681,19.624101,0.658744,1.282648,1.133571,0.975080,DX-Y.NYB
7,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,10001.762815,0.017628,-0.777941,100.0,312.933015,4.244304,...,-0.513340,19 days 00:00:00,2 days 21:49:05.454545454,1.009488,0.360560,0.031110,0.005992,1.006076,0.043643,DX=F
8,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,10047.815427,0.478154,7.162079,100.0,405.791012,10.558327,...,-1.060417,15 days 00:00:00,2 days 18:27:41.538461538,1.040012,3.045434,0.117552,0.065404,1.023870,0.158559,ES=F
9,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,10924.353942,9.243539,39.454500,100.0,494.110255,8.350617,...,-1.055848,12 days 00:00:00,2 days 01:42:51.428571428,1.468432,31.716020,0.969760,1.629085,1.185881,1.313356,GC=F


In [2]:
df = pd.read_csv("vectorbt_reports/rolling_master_backtest_stats.csv")
display(df)

,Start,End,Period,Start Value,End Value,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],...,Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,asset
0,2024-06-03,2025-04-01,208 days,10000.0,8989.263065,-10.107369,15.028606,100.0,379.736994,20.505496,...,-2.527041,9 days 08:00:00,3 days 12:55:23.076923076,0.591091,-69.592851,-0.733818,-0.831687,0.866756,-0.922554,AAPL
1,2024-06-03 00:00:00,2025-04-01 00:00:00,208 days 00:00:00,10000.0,7685.117277,-23.148827,7.754851,100.0,418.130914,26.313659,...,-2.643580,12 days 04:48:00,3 days 09:20:00,0.464750,-100.312939,-1.463967,-1.406124,0.766458,-1.836301,AMZN
2,2024-06-01 00:00:00,2025-04-02 00:00:00,306 days 00:00:00,10000.0,8441.518228,-15.584818,26.265288,100.0,694.273859,40.259704,...,-3.272167,9 days 00:00:00,4 days 04:00:00,0.812415,-48.702555,-0.378473,-0.454494,0.929393,-0.554446,BTC-USD
3,2024-06-03 00:00:00,2025-04-01 00:00:00,209 days 00:00:00,10000.0,10086.190920,0.861909,22.523686,100.0,424.992020,28.885195,...,-2.880341,7 days 00:00:00,4 days 09:13:50.769230769,1.022573,4.104330,0.273277,0.052279,1.054952,0.415850,BTC=F
4,2024-06-02 00:00:00,2025-04-02 00:00:00,217 days 00:00:00,10000.0,9481.452002,-5.185480,2.058563,100.0,452.203626,8.675658,...,-0.607394,7 days 19:12:00,4 days 09:20:00,0.518455,-22.545565,-1.238039,-0.987478,0.802285,-1.728193,CHFUSD=X
5,2024-06-03 00:00:00,2025-04-01 00:00:00,209 days 00:00:00,10000.0,6794.355080,-32.056449,-4.068990,100.0,419.095957,41.015994,...,-2.710588,7 days 04:48:00,2 days 12:00:00,0.217616,-143.390314,-2.669798,-1.196683,0.596888,-3.254813,CL=F
6,2024-06-03 00:00:00,2025-04-01 00:00:00,209 days 00:00:00,10000.0,9075.636280,-9.243637,0.115232,100.0,465.682626,10.174957,...,-0.621104,6 days 00:00:00,3 days 03:47:22.105263157,0.121628,-41.150434,-3.040811,-1.531388,0.566349,-3.558531,DX-Y.NYB
7,2024-06-03 00:00:00,2025-04-01 00:00:00,209 days 00:00:00,10000.0,8942.438886,-10.575611,-0.123951,100.0,478.160345,11.312000,...,-0.686864,7 days 04:48:00,3 days 00:00:00,0.134582,-44.893247,-3.183677,-1.567686,0.554201,-3.740556,DX=F
8,2024-06-03 00:00:00,2025-04-01 00:00:00,209 days 00:00:00,10000.0,8792.170141,-12.078299,7.121620,100.0,397.855981,16.038483,...,-1.869440,8 days 13:42:51.428571428,5 days 05:32:18.461538461,0.497455,-60.391493,-1.700318,-1.255277,0.739663,-2.077583,ES=F
9,2024-06-03 00:00:00,2025-04-01 00:00:00,209 days 00:00:00,10000.0,9719.151187,-2.808488,32.911437,100.0,503.408472,10.170212,...,-1.035418,7 days 09:36:00,2 days 16:56:28.235294117,0.831407,-10.401808,-0.271456,-0.477202,0.954273,-0.343202,GC=F


In [3]:
df = pd.read_csv("vectorbt_reports/new_expanding_master_backtest_stats.csv")
display(df)

,Start,End,Period,Start Value,End Value,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],...,Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,asset
0,2024-06-03,2025-04-01,208 days,10000.0,10428.952729,4.289527,15.028606,100.0,348.922817,10.968799,...,-2.293068,9 days 03:25:42.857142857,3 days 18:40:00,1.030456,4.220772,0.453001,0.697317,1.092223,0.611890,AAPL
1,2024-06-03 00:00:00,2025-04-01 00:00:00,208 days 00:00:00,10000.0,7392.356184,-26.076438,7.754851,100.0,373.120125,28.449814,...,-2.945364,11 days 04:48:00,3 days 12:00:00,0.376853,-123.821498,-1.756198,-1.446441,0.713576,-2.156633,AMZN
2,2024-06-01 00:00:00,2025-04-02 00:00:00,306 days 00:00:00,10000.0,7095.941435,-29.040586,26.265288,100.0,595.804773,44.711245,...,-3.467318,10 days 00:00:00,3 days 13:33:54.782608695,0.619954,-93.679309,-0.990917,-0.751095,0.821695,-1.400476,BTC-USD
3,2024-06-03 00:00:00,2025-04-01 00:00:00,209 days 00:00:00,10000.0,8711.523782,-12.884762,22.523686,100.0,434.194664,27.568969,...,-3.065993,8 days 00:00:00,4 days 00:00:00,0.711743,-61.356010,-0.407550,-0.776516,0.930398,-0.601356,BTC=F
4,2024-06-02 00:00:00,2025-04-02 00:00:00,217 days 00:00:00,10000.0,9165.462230,-8.345378,2.058563,100.0,480.617994,10.151395,...,-0.577005,9 days 00:00:00,4 days 00:00:00,0.317234,-33.381511,-2.092640,-1.343071,0.687246,-2.810987,CHFUSD=X
5,2024-06-03 00:00:00,2025-04-01 00:00:00,209 days 00:00:00,10000.0,7116.052405,-28.839476,-4.068990,100.0,394.548319,36.288358,...,-2.842522,7 days 12:00:00,2 days 07:03:31.764705882,0.203394,-142.652638,-2.350944,-1.234522,0.634843,-2.910320,CL=F
6,2024-06-03 00:00:00,2025-04-01 00:00:00,209 days 00:00:00,10000.0,8748.491465,-12.515085,0.115232,100.0,529.436606,13.053818,...,-0.590670,5 days 16:00:00,3 days 05:45:36,0.076068,-45.614214,-4.120344,-1.595272,0.477101,-4.674413,DX-Y.NYB
7,2024-06-03 00:00:00,2025-04-01 00:00:00,209 days 00:00:00,10000.0,8723.553756,-12.764462,-0.123951,100.0,510.583461,13.341226,...,-0.632384,5 days 16:00:00,3 days 07:00:00,0.080348,-48.358992,-3.963554,-1.590417,0.482922,-4.483572,DX=F
8,2024-06-03 00:00:00,2025-04-01 00:00:00,209 days 00:00:00,10000.0,8662.327129,-13.376729,7.121620,100.0,431.828063,17.347796,...,-1.745900,8 days 20:34:17.142857142,4 days 16:00:00,0.478190,-60.803312,-1.856384,-1.278620,0.722039,-2.242550,ES=F
9,2024-06-03 00:00:00,2025-04-01 00:00:00,209 days 00:00:00,10000.0,9848.774568,-1.512254,32.911437,100.0,488.911068,9.443533,...,-1.089100,8 days 10:40:00,2 days 15:31:45.882352941,0.914104,-5.816363,-0.109137,-0.278084,0.981437,-0.139356,GC=F


In [4]:
df = pd.read_csv("backtest_data/TSLA.csv")
display(df)

,Date,CPI_XYOY_INDEX,CPI_YOY_INDEX,PCE_CYOY_INDEX,PCE_YOY_INDEX,PPI_YOY_INDEX,GDP_CQOQ_INDEX,IP_INDEX,NAPMNMI_INDEX,NAPMPMI_INDEX,...,CPI_Unemployment,GDP_RetailSales,Sentiment_Retail,YieldSpread_Fed,CPI_minus_FedRate,GDP_Unemployment,Yield_Curve_10y_2y,Yield_Curve_10y_3m,Real_Rate_Proxy,label
0,2024-04-01,0.308607,3.086067e-01,3.086067e-01,3.086067e-01,-3.086067e-01,-4.440892e-16,3.086067e-01,0.308607,0.308607,...,3.086067e-01,3.086067e-01,0.308607,-1.474400,3.086067e-01,0.00,-1.474400,-1.876544,-1.718814,1.0
1,2024-04-02,3.800000,3.500000e+00,2.979610e+00,5.600000e+00,1.800000e+00,1.600000e+00,1.025186e+02,51.300000,49.800000,...,1.365000e+01,6.400000e-01,31.760000,-187.962500,-2.000000e+00,-4.64,-34.175000,-102.520000,0.849100,NaN
2,2024-04-03,3.800000,3.500000e+00,2.979610e+00,5.600000e+00,1.800000e+00,1.600000e+00,1.025186e+02,51.300000,49.800000,...,1.365000e+01,6.400000e-01,31.760000,-179.949000,-2.000000e+00,-4.64,-32.718000,-102.373000,0.847200,NaN
3,2024-04-04,3.800000,3.500000e+00,2.979610e+00,5.600000e+00,1.800000e+00,1.600000e+00,1.025186e+02,51.300000,49.800000,...,1.365000e+01,6.400000e-01,31.760000,-187.071500,-2.000000e+00,-4.64,-34.013000,-105.814000,0.809400,NaN
4,2024-04-05,3.800000,3.500000e+00,2.979610e+00,5.600000e+00,1.800000e+00,1.600000e+00,1.025186e+02,51.300000,49.800000,...,1.365000e+01,6.400000e-01,31.760000,-193.044500,-2.000000e+00,-4.64,-35.099000,-98.182000,0.901600,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2025-03-26,3.100000,2.800000e+00,2.788710e+00,5.300000e+00,2.100000e+00,2.400000e+00,1.042062e+02,53.500000,50.300000,...,1.148000e+01,4.800000e-01,12.940000,148.968000,-1.700000e+00,-7.44,33.104000,5.495000,1.551900,NaN
248,2025-03-27,3.100000,2.800000e+00,2.788710e+00,5.300000e+00,2.100000e+00,2.400000e+00,1.042062e+02,53.500000,50.300000,...,1.148000e+01,4.800000e-01,12.940000,164.497500,-1.700000e+00,-7.44,36.555000,6.120000,1.559500,NaN
249,2025-03-28,3.100000,2.800000e+00,2.788710e+00,5.300000e+00,2.100000e+00,2.400000e+00,1.042062e+02,53.500000,50.300000,...,1.148000e+01,4.800000e-01,12.940000,150.952500,-1.700000e+00,-7.44,33.545000,-4.743000,1.449400,NaN
250,2025-03-31,3.100000,2.800000e+00,2.788710e+00,5.300000e+00,2.100000e+00,2.400000e+00,1.042062e+02,53.500000,49.000000,...,1.148000e+01,4.800000e-01,11.400000,143.095500,-1.700000e+00,-7.44,31.799000,-9.883000,1.405300,NaN


In [ ]:
df = pd.read_csv("vectorbt_reports/fixed_master_backtest_stats.csv")
display(df)